In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader

def load_resume(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

d:\The-Office\Projects\RAG\cold-email\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

def load_jobs(path="data/jobs.csv"):
    return pd.read_csv(path)

In [3]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

def match_job_to_resume(job_description, resume_text, k = 5):
    job_emb = model.encode(job_description, convert_to_tensor=True)
    resume_emb = model.encode(resume_text, convert_to_tensor=True)
    cos_scores = util.cos_sim(job_emb, resume_emb)[0]
    top_results = cos_scores.topk(k)
    return top_results

In [4]:
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def generate_email(resume, job):
    prompt = f"""
Write a short, sharp cold email (<120 words) applying for this role: {job['job_title']}.

My resume summary:
{resume}

Job description:
{job['job_description']}

Email must:
- be 5–6 sentences
- highlight relevant skills
- sound confident and professional
- end with a call to action
"""

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [ ]:
import yagmail

def send_email(to, subject, body, attachment=None):
    EMAIL = os.getenv("EMAIL")
    APP_PASSWORD = os.getenv("APP_PASSWORD")
    yag = yagmail.SMTP(EMAIL,APP_PASSWORD)

    yag.send(to=to, subject=subject, contents=body, attachments=attachment)

In [10]:
resume = load_resume("../data/resume.pdf")
jobs = load_jobs("../data/jobs.csv")
matched = match_job_to_resume(jobs.iloc[0]['job_description'], resume[0].page_content, k=1)
print(matched)
job = jobs.iloc[matched[1][0].item()]
print(job)

torch.return_types.topk(
values=tensor([0.3910]),
indices=tensor([0]))
job_title                                        Data Analyst Intern
job_description    We are looking for an intern with strong analy...
hr_email                                              hr@company.com
Name: 0, dtype: object


In [11]:
email_content = generate_email(resume[0].page_content, job)

In [16]:
from dotenv import load_dotenv
load_dotenv()
TEST_EMAIL = os.getenv("TEST_EMAIL")
attachment_path = "../data/resume_x.pdf"
send = input("Send email? (y/n): ")
if send.lower() == 'y':
    send_email(
        #to=job['contact_email'],
        to=TEST_EMAIL,
        subject=f"Application for {job['job_title']}",
        body=email_content,
        attachment=attachment_path
    )
    print("Email sent!")

../data/resume_x.pdf
Email sent!
